## Table of Contents

1. Create datafrme from wikipage  and clean the data   
2. Obtain the Longitude and Latitude
3. Analyze the Toronto neighborhood and prepare the data for clustering
4. Cluster the Toronto neighborhood


### 1. Create dataframe from wikipage and clean the data:
- Obtain the table,
- Drop the rows where Borough is 'Not assigned',
- Find out duplicated values in Postal Code column and combine the value in Neighborhood column,
- Find out the the rows where Borough is assigned where Neighborhood is not, assign the value of Neighbohood from Borough.

#### Obtain the table,
- Initial output (df) is a list of list, 
- Extract the first element of the list and convert it to dataframe df

In [22]:
import pandas as pd
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" , header = 0)
df = pd.DataFrame(df[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code     180 non-null object
Borough         180 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


#### Drop the rows where Borough is "Not assigned"

In [23]:
df = df[df.Borough != 'Not assigned']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 3 columns):
Postal Code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB


#### Find out duplicated values in Postal Code column and combine the value in Neighborhood column
- No duplicated values found in Postal Code column

In [24]:
duplicateRowsdf = df[df['Postal Code'].duplicated()]
print(duplicateRowsdf)

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
- No cell found

In [25]:
df[df['Neighborhood']== '']

,Postal Code,Borough,Neighborhood


In [26]:
df.shape

(103, 3)

### 2. Obtain the Longitude and Lantitude
- Using Geocoder package
- Using Geospatial_data

#### Using Geocoder package
- Initial a testing to see if the package work
- request denied by Google

In [27]:
import geocoder
g = geocoder.google('{}, Toronto, Ontario')
g

<[REQUEST_DENIED] Google - Geocode [empty]>

#### Using Geospatial_date
- Import the file in dataframe: Geo_df
- checkout the head and dataframe information
- Inner join the df and Geo_df by Postal Code column as key to obtain the longitute and lantitude

In [28]:
Geo_df = pd.read_csv('Geospatial_Coordinates.csv')

In [29]:
Geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
Geo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [31]:
merge_df = pd.merge(df, Geo_df, how = 'inner', on = 'Postal Code')

In [32]:
merge_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [33]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
Postal Code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [34]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 3. Cluster the Toronto neighborhoods and prepare the data for clusering
- Visualize the Toronto neighborhood
- Define FourSquare Credential
- Get all the venues of Toronto neighborhood and create the a new dataframe tot_venues
- Check the size of tot_venues
- Analyze Each Neighborhood

#### Visualize the Toronto neighborhod
- Find out all boroughs that contains 'Toronto'
- Get the geographic coordinates of Toronto
- Visualize the Toronto neighborhood

Find out all the borough that contains 'Toronto'

In [35]:
tot_data = merge_df[merge_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
tot_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Get the gerographic coordinates of Toronto

In [36]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tot_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Visualize the Toronto neighborhood

In [37]:
# create map of Manhattan using latitude and longitude values
map_tot = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(tot_data['Latitude'], tot_data['Longitude'], tot_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tot)  
    
map_tot

#### Define FourSquare Credential

In [38]:
CLIENT_ID = '5VNESGAO30KXIF03MJ5GKIIXK3NGTUAPEETYHULZQOYNDFC3' # your Foursquare ID
CLIENT_SECRET = 'VTCZNXWWWESYCRIYJEALHEMNZZR1ZP4UB430L5HCETB5HYCF' # your Foursquare Secret
VERSION = '20200516' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5VNESGAO30KXIF03MJ5GKIIXK3NGTUAPEETYHULZQOYNDFC3
CLIENT_SECRET:VTCZNXWWWESYCRIYJEALHEMNZZR1ZP4UB430L5HCETB5HYCF


#### Get all the venues of Toronto neighborhood and create a new dataframe tot_venues

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
tot_venues = getNearbyVenues(names=tot_data['Neighborhood'],
                                   latitudes=tot_data['Latitude'],
                                   longitudes=tot_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

#### Check the size of new dataframe
- Check the number of venues of each neighborhood
- unique categories

In [41]:
print(tot_venues.shape)
tot_venues.head()

(1602, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [42]:
tot_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
Business reply mail Processing Centre,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [43]:
print('There are {} uniques categories.'.format(len(tot_venues['Venue Category'].unique())))

There are 235 uniques categories.


#### Analyze the dataframe
- Use one-hot coding to convert categorical variable (venue) into binary variable
- Group by the neighborhood using the average frequency of each venue
- Find out the top 40 venues and for each neighbor and conver into data frame

One-hot coding

In [109]:
# one hot encoding
tot_onehot = pd.get_dummies(tot_venues[['Venue Category']], prefix="", prefix_sep="")
# Add and move neighborhood column to the first column, since the category contains 'Neighborhood', the first column is renamed
# into 'Neighborhood_idx'
tot_onehot.insert(loc=0, column='Neighborhood_idx', value=tot_venues['Neighborhood'])

tot_onehot.head()

,Neighborhood_idx,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [45]:
tot_onehot.shape

(1602, 236)

Group by the neighborhood

In [47]:
tot_grouped = tot_onehot.groupby('Neighborhood_idx').mean().reset_index()
tot_grouped

,Neighborhood_idx,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.0000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.000000,0.035714,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.017857,0.035714,0.000000,0.0000,0.000000,0.035714,0.000000,0.000000,0.000000,0.00,0.017857,0.053571,0.071429,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.017857,0.000000,0.017857,0.000,0.00,0.000000,0.000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [48]:
tot_grouped.shape

(39, 236)

The top 40 venues for each neighborhood

In [101]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [102]:
num_top_venues = 40

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tot_grouped['Neighborhood_idx']

for ind in np.arange(tot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Café,Cheese Shop,Seafood Restaurant,Bakery,Jazz Club,Shopping Mall,Eastern European Restaurant,Bistro,Breakfast Spot,Hotel,Irish Pub,Farmers Market,Japanese Restaurant,Grocery Store,Pub,Beach,Butcher,Diner,Liquor Store,Park,Clothing Store,Nightclub,Creperie,Museum,Comfort Food Restaurant,Juice Bar,Concert Hall,Art Gallery,Bagel Shop,Thai Restaurant,Greek Restaurant,Basketball Stadium,Gourmet Shop,French Restaurant,Tailor Shop,Vegetarian / Vegan Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Yoga Studio,Gym,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection,Gym / Fitness Center,Convenience Store,Furniture / Home Store,Stadium,Climbing Gym,Burrito Place,Bar,Bakery,Restaurant,Grocery Store,Sushi Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Toy / Game Store,Concert Hall,Comic Shop,Comfort Food Restaurant,Colombian Restaurant,College Rec Center
2,Business reply mail Processing Centre,Brewery,Smoke Shop,Recording Studio,Fast Food Restaurant,Farmers Market,Auto Workshop,Burrito Place,Restaurant,Pizza Place,Butcher,Garden,Garden Center,Gym / Fitness Center,Comic Shop,Light Rail Station,Park,Skate Park,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Yoga Studio,Cupcake Shop,Deli / Bodega,Dance Studio,Doner Restaurant,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Concert Hall,Comfort Food Restaurant,Colombian Restaurant,Dog Run,Eastern European Restaurant,Donut Shop,Food Truck
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport,Airport Food Court,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,General Entertainment,Event Space,Dance Studio,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Concert Hall,Comic Shop,Deli / Bodega,Falafel Restaurant,General Travel,Fountain
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Thai Restaurant,Japanese Restaurant,Bar,Ice Cream Shop,Burger Joint,Salad Place,Bubble Tea Shop,Ramen Restaurant,Discount Store,Donut Shop,Bank,Diner,Indian Restaurant,Poke Place,Hotel,Spa,Falafel Restaurant,Department Store,Korean Restaurant,Yoga Studio,Restaurant,Middle Eastern Restaurant,French Restaurant,Gastropub,Sushi Restaurant,Comic Shop,Park,Gym / Fitness Center,Art Museum,Vegetarian / Vegan Restaurant,Modern European Restaurant,Office,Concert Hall,Convenience Store,Cupcake Shop,Convention Center
5,Christie,Grocery Store,Café,Park,Restaurant,Coffee Shop,Nightclub,Diner,Candy Store,Italian Restaurant,Baby Store,Deli 

### 4. Cluster Toronto Neighborhoods
- Using K means to cluster the neighborhood into 10 clusters
- Visulize the result

Run *k*-means to cluster the neighborhood into 10 clusters.

In [110]:
# set number of clusters
kclusters = 10

tot_grouped_clustering = tot_grouped.drop('Neighborhood_idx', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 0, 1, 4, 4, 4, 4, 8])

In [111]:
# add clustering labels
#neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop(['Cluster Labels'], axis = 1)
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis = 1, inplace = True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tot_merged = tot_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tot_merged = tot_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tot_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Mexican Restaurant,Shoe Store,Restaurant,Chocolate Shop,Cosmetics Shop,Dessert Shop,Spa,Distribution Center,Electronics Store,Event Space,Farmers Market,Ice Cream Shop,Hotel,French Restaurant,Historic Site,Gym / Fitness Center,Beer Store,Performing Arts Venue,Asian Restaurant,Bank,Antique Shop,Art Gallery,Fast Food Restaurant,Airport Terminal,Falafel Restaurant,American Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Beach
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,College Cafeteria,Creperie,Burrito Place,Distribution Center,Japanese Restaurant,Smoothie Shop,Italian Restaurant,Beer Bar,Diner,Café,Mexican Restaurant,Bar,Bank,Yoga Studio,Hobby Shop,Arts & Crafts Store,College Auditorium,Gym,Park,Theater,Fried Chicken Joint,Sandwich Place,Department Store,Coworking Space,Dessert Shop,Deli / Bodega,Cuban Restaurant,Convention Center,Discount Store,Eastern European Restaurant,Dance Studio,Cupcake Shop,Convenience Store,Concert Hall,Dog Run,Doner Restaurant,Donut Shop,Cosmetics Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Clothing Store,Coffee Shop,Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Tea Room,Hotel,Ramen Restaurant,Fast Food Restaurant,Bookstore,Theater,Lingerie Store,Diner,Burrito Place,Pub,Poutine Place,Plaza,Hookah Bar,Department Store,Dessert Shop,Pizza Place,Burger Joint,New American Restaurant,Park,Other Great Outdoors,Office,Sandwich Place,Music Venue,Movie Theater,Comic Shop,Chinese Restaurant,Modern European Restaurant,Miscellaneous Shop,Mexican Restaurant,Lounge,College Rec Center
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Department Store,Italian Restaurant,Restaurant,Beer Bar,Clothing Store,Moroccan Restaurant,Cosmetics Shop,Creperie,Lingerie Store,Seafood Restaurant,Gym,Theater,Irish Pub,Salon / Barbershop,Breakfast Spot,French Restaurant,Burrito Place,German Restaurant,Fountain,Pharmacy,Park,Food Truck,Fish Market,Japanese Restaurant,New American Restaurant,Farmers Market,Comfort Food Restaurant,Middle Eastern Restaurant,Hotel,Ice Cream Shop,Bookstore,Diner,Jazz Club,Grocery Store,Furniture / Home Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,7,Trail,Neighborhood,Pub,Health Food Store,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio,Dance Studio,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Deli / Bodega,Creperie,Cupcake Shop,Cuban Restaurant,Event Space,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Concert Hall,Comic Shop,Comfort Food Restaurant,Colombian Restaurant,College Rec Center,Ethiopian Restaurant,Falaf

visualize the resulting clusters

In [112]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tot_merged['Latitude'], tot_merged['Longitude'], tot_merged['Neighborhood'], tot_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [113]:
Label_sum = tot_merged.groupby('Cluster Labels')['Postal Code'].count().reset_index()
Label_sum['Percentage'] = (Label_sum['Postal Code'] / Label_sum['Postal Code'].sum()).round(decimals = 2)
Label_sum = Label_sum.sort_values(by = ['Postal Code'], ascending = False)
Label_sum

,Cluster Labels,Postal Code,Percentage
4,4,27,0.69
1,1,4,0.10
0,0,1,0.03
2,2,1,0.03
3,3,1,0.03
5,5,1,0.03
6,6,1,0.03
7,7,1,0.03
8,8,1,0.03
9,9,1,0.03


### Summary
- The largest cluster: Label 4 with 27 neighborhoods, consisting 69% of total neighborhoods

In [116]:
tot_merged[tot_merged['Cluster Labels'] == 4]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Clothing Store,Coffee Shop,Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Tea Room,Hotel,Ramen Restaurant,Fast Food Restaurant,Bookstore,Theater,Lingerie Store,Diner,Burrito Place,Pub,Poutine Place,Plaza,Hookah Bar,Department Store,Dessert Shop,Pizza Place,Burger Joint,New American Restaurant,Park,Other Great Outdoors,Office,Sandwich Place,Music Venue,Movie Theater,Comic Shop,Chinese Restaurant,Modern European Restaurant,Miscellaneous Shop,Mexican Restaurant,Lounge,College Rec Center
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Department Store,Italian Restaurant,Restaurant,Beer Bar,Clothing Store,Moroccan Restaurant,Cosmetics Shop,Creperie,Lingerie Store,Seafood Restaurant,Gym,Theater,Irish Pub,Salon / Barbershop,Breakfast Spot,French Restaurant,Burrito Place,German Restaurant,Fountain,Pharmacy,Park,Food Truck,Fish Market,Japanese Restaurant,New American Restaurant,Farmers Market,Comfort Food Restaurant,Middle Eastern Restaurant,Hotel,Ice Cream Shop,Bookstore,Diner,Jazz Club,Grocery Store,Furniture / Home Store
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Café,Cheese Shop,Seafood Restaurant,Bakery,Jazz Club,Shopping Mall,Eastern European Restaurant,Bistro,Breakfast Spot,Hotel,Irish Pub,Farmers Market,Japanese Restaurant,Grocery Store,Pub,Beach,Butcher,Diner,Liquor Store,Park,Clothing Store,Nightclub,Creperie,Museum,Comfort Food Restaurant,Juice Bar,Concert Hall,Art Gallery,Bagel Shop,Thai Restaurant,Greek Restaurant,Basketball Stadium,Gourmet Shop,French Restaurant,Tailor Shop,Vegetarian / Vegan Restaurant
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Restaurant,Coffee Shop,Nightclub,Diner,Candy Store,Italian Restaurant,Baby Store,Deli / Bodega,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Dessert Shop,Department Store,Comfort Food Restaurant,Dance Studio,Comic Shop,Electronics Store,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Concert Hall,Colombian Restaurant,Event Space,Ethiopian Restaurant,Fried Chicken Joint,German Restaurant,General Travel,General Entertainment,Gay Bar,Gastropub
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,4,Coffee Shop,Café,Restaurant,Clothing Store,Deli / Bodega,Hotel,Thai Restaurant,Gym,Bookstore,Sushi Restaurant,Cosmetics Shop,Pizza Place,Salad Place,Seafood Restaurant,Steakhouse,Concert Hall,American Restaurant,Gastropub,Breakfast Spot,Pub,Asian Restaurant,Burger Joint,Poke Place,Plaza,Burrito Place,Vegetarian / Vegan Restaurant,Ice Cream Shop,Furniture / Home Store,Opera House,Cupcake Shop,Nightclub,New American Restaurant,Neighborhood,Japanese Restaurant,Monument / Landmark,Modern European Restaurant